In [3]:
# establish connection
from os import getenv
from dotenv import load_dotenv
import pyodbc
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=localhost\\DOKUSSERVER;"
                      "Database=Lernperiode12;"
                      "Trusted_Connection=yes;")

cursor = cnxn.cursor()

In [21]:
# plant template
SQL_STATEMENT = """
INSERT Plant (
name,
location
) VALUES
( ?, ?)
"""

In [23]:
# insert plant data
cursor.execute(
    SQL_STATEMENT,
    (f'Neverdie',
     f'By window'
    )
)

cnxn.commit()

In [3]:
# select data from db
cursor.execute("SELECT * FROM Plant")
rows = cursor.fetchall()
for r in rows:
    print(r)

(1, 'Neverdie', 'By window')
(2, 'Neverdie', 'By window')


In [4]:
# import for request
import requests
import json
from datetime import datetime

def get_weather_data(latitude, longitude):
    base_url = "https://api.open-meteo.com/v1/forecast"
    
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "current": ["temperature_2m", "relative_humidity_2m", "precipitation", "wind_speed_10m", "cloud_cover"],
        "timezone": "auto"
    }
    # Make the API request
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes
        
        data = response.json()
        
        current = data["current"]
        
        print("\nCurrent Weather Information:")
        print(f"Temperature: {current['temperature_2m']}°C")
        print(f"Humidity: {current['relative_humidity_2m']}%")
        print(f"Precipitation: {current['precipitation']} mm")
        print(f"Wind Speed: {current['wind_speed_10m']} km/h")
        print(f"Last Updated: {current['time']}")
        
        return data
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching weather data: {e}")
        return None


In [5]:
# Coordinates for Baden, Switzerland
baden_lat = 47.475361
baden_lon = 8.306372

print("Fetching weather data for Baden...")
weather_data = get_weather_data(baden_lat, baden_lon)

if weather_data:
    # If you want to see the raw JSON data
    print("\nRaw weather data:")
    print(json.dumps(weather_data, indent=2))

Fetching weather data for Baden...

Current Weather Information:
Temperature: 2.8°C
Humidity: 81%
Precipitation: 0.0 mm
Wind Speed: 3.1 km/h
Last Updated: 2025-11-28T14:30

Raw weather data:
{
  "latitude": 47.48,
  "longitude": 8.299999,
  "generationtime_ms": 0.0782012939453125,
  "utc_offset_seconds": 3600,
  "timezone": "Europe/Zurich",
  "timezone_abbreviation": "GMT+1",
  "elevation": 394.0,
  "current_units": {
    "time": "iso8601",
    "interval": "seconds",
    "temperature_2m": "\u00b0C",
    "relative_humidity_2m": "%",
    "precipitation": "mm",
    "wind_speed_10m": "km/h",
    "cloud_cover": "%"
  },
  "current": {
    "time": "2025-11-28T14:30",
    "interval": 900,
    "temperature_2m": 2.8,
    "relative_humidity_2m": 81,
    "precipitation": 0.0,
    "wind_speed_10m": 3.1,
    "cloud_cover": 100
  }
}


In [6]:
print(weather_data)

{'latitude': 47.48, 'longitude': 8.299999, 'generationtime_ms': 0.0782012939453125, 'utc_offset_seconds': 3600, 'timezone': 'Europe/Zurich', 'timezone_abbreviation': 'GMT+1', 'elevation': 394.0, 'current_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature_2m': '°C', 'relative_humidity_2m': '%', 'precipitation': 'mm', 'wind_speed_10m': 'km/h', 'cloud_cover': '%'}, 'current': {'time': '2025-11-28T14:30', 'interval': 900, 'temperature_2m': 2.8, 'relative_humidity_2m': 81, 'precipitation': 0.0, 'wind_speed_10m': 3.1, 'cloud_cover': 100}}


In [7]:
# weather data template
SQL_API_STATEMENT = """
INSERT INTO WeatherMeasurement (
    timestamp,
    temperature,
    humidity,
    windspeed,
    precipitation
) VALUES
( ?, ?, ?, ?, ?)
"""


In [8]:
current_weather = weather_data["current"]
print(current_weather["temperature_2m"])

2.8


In [9]:
# insert current weather data
from datetime import datetime

dt = datetime.fromisoformat(current_weather["time"])


cursor.execute(
    SQL_API_STATEMENT,
    (dt,
     current_weather["temperature_2m"],
     current_weather["relative_humidity_2m"],
     current_weather["wind_speed_10m"],
     current_weather["precipitation"],
    )
)
cnxn.commit()

In [24]:
# select data from db
cursor.execute("SELECT * FROM WeatherMeasurement")
rows = cursor.fetchall()
for r in rows:
    print(r)

(2, datetime.datetime(2025, 11, 21, 16, 30), 0.5, 67.0, 11.9, 0.0)
(3, datetime.datetime(2025, 11, 28, 14, 30), 2.8, 81.0, 3.1, 0.0)


In [20]:
cursor.execute("SELECT MAX(timestamp) FROM WeatherMeasurement")

In [21]:
rows = cursor.fetchall()
for r in rows:
    print(r)

(datetime.datetime(2025, 11, 28, 14, 30),)


In [8]:
last_date = cursor.fetchall()
print(last_date)

TypeError: Cursor.fetchall() takes no arguments (1 given)

In [16]:
cursor.execute("DELETE FROM WeatherMeasurement WHERE weatherID > 3")

In [14]:
cursor.execute("SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'WeatherMeasurement'")

In [17]:
print(cursor.fetchall())

ProgrammingError: No results.  Previous SQL was not a query.

In [18]:
cnxn.commit()